# Näidisnäide Hotelli ja Lennukipileti Broneerimise Agent

See lahendus aitab teil broneerida lennupileteid ja hotelli. Stsenaarium on reis London Heathrow LHR 20. veebruar 2024 New York JFK, tagasireis 27. veebruar 2025, majanduskabinetiga ainult British Airwaysiga. Soovin ööbimist Hiltoni hotellis New Yorgis, palun esitage lennu ja hotelli hinnad.


# Azure AI agendi teenuse initsialiseerimine ja konfiguratsiooniinfo saamine failist **.env**

### **.env**

Loo .env fail

**.env** sisaldab Azure AI agendi teenuse ühendusstringi, AOAI poolt kasutatava mudeli ja vastava Google API Search teenuse API, lõpp-punkti jne.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Teie Azure AI agendi teenuse mudeli juurutamise nimi"

[**MÄRKUS**] Teil on vaja mudelit, millel on 100,000 piirarv (märke minutis) ja piirarv 600 (päringut minutis)

  Mudelit saate Microsoft Foundrys - Mudel ja Lõpp-punkt.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Teie Azure AI agendi teenuse projekti ühendusstring"

  Projekti ühendusstringi leiate oma projekti ülevaatest AI Foundry portaali ekraanil.

- **SERPAPI_SEARCH_API_KEY** = "Teie SERPAPI otsingu API VÕTI"
- **SERPAPI_SEARCH_ENDPOINT** = "Teie SERPAPI otsingu lõpp-punkt"

Azure AI agendi teenuse mudeli juurutamise nime ja projekti ühendusstringi saamiseks peate looma Azure AI agendi teenuse. Soovitatav on kasutada [seda mall](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) selle otse loomiseks （***Märkus:*** Azure AI agendi teenus on hetkel seadistatud piiratud piirkonnas. Soovitatav on viidata [sellele lingile](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) piirkonna seadistamiseks)

Agent vajab juurdepääsu SERPAPI-le. Soovitatav on registreeruda kasutades [seda linki](https://serpapi.com/searches). Pärast registreerumist saate unikaalse API VÕTI ja LÕPP-PUNKTi.


# Seadistamine

Selle märkmiku käivitamiseks peate veenduma, et olete paigaldanud vajalikud raamatukogud, käivitades käsu `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Teie Semantic Kernel versioon peaks olema vähemalt 1.27.2.


Laadige oma .env faili seadistused ja ressursid, palun veenduge, et olete lisanud oma võtmed ja seadistused ning loonud kohaliku .env faili.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Logi sisse Azure'i

Nüüd peate Azure'i sisse logima. Avage terminal ja käivitage järgmine käsk:

```bash
az login
```

See käsk palub teil sisestada oma Azure'i mandaadid, võimaldades Azure AI Agendi teenusel õigesti toimida.


# Selgitus:
See on muutuja, mis hoiab SERP (otsingumootori tulemuste lehekülje) API-teenuse kasutamiseks vajalikku API-võtit. API-võti on unikaalne identifikaator, mida kasutatakse päringute autentimiseks, mis on seotud teie kontoga.

Eesmärk: Selle rea eesmärk on hoida API-võtit muutuvas, et seda saaks kasutada päringute autentimiseks SERP API-teenuses. API-võti on teenuse kasutamiseks ja otsingute tegemiseks kohustuslik.

Kuidas saada SERP API-võti: SERP API-võtme saamiseks järgige üldisi samme saidil https://serpapi.com (täpsed sammud võivad varieeruda sõltuvalt kasutatavast konkreetsest SERP API-teenusest):

Valige SERP API teenus: Saadaval on mitmeid SERP API teenuseid, nagu SerpAPI, Google Custom Search JSON API ja teised. Valige see, mis kõige paremini vastab teie vajadustele.

Registreeruge konto saamiseks: Minge valitud SERP API teenuse veebisaidile ja registreeruge konto loomiseks. Võimalik, et peate esitama mõned põhiandmed ja kinnitama oma e-posti aadressi.

Looge API võti: Pärast registreerumist logige kontole sisse ja minge API sektsiooni või juhtpaneelile. Otsige võimalust luua või genereerida uus API-võti.
Kopeerige API-võti oma .env faili.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Selgitus:
BASE_URL: See on muutuja, mis salvestab SERP API lõpp-punkti baas-URL-i. Muutuja nimi BASE_URL on tava, mida kasutatakse selleks, et näidata, et see URL on alguspunktiks API päringute tegemisel.
'https://serpapi.com/search':

See on tegelik URL-i string, mis on määratud muutujale BASE_URL. See esindab lõpp-punkti, mida kasutatakse otsingupäringute tegemiseks SERP API abil.

# Eesmärk:
Selle rea eesmärk on defineerida konstant, mis hoiab SERP API baas-URL-i. Seda URL-i kasutatakse API päringute koostamise alguspunktina otsingute teostamiseks.

# Kasutus:
Baas-URL-i muutujana määrates saate seda kergesti taaskasutada kogu oma koodis igal ajal, kui on vaja teha päringuid SERP API-le. See muudab teie koodi paremini hallatavaks ja vähendab vigade tekkimise riski, mis võib tekkida URL-i kõvakodeerimisel mitmesse kohta. Praegune näide on https://serpapi.com/search?engine=bing, mis kasutab Bing'i otsingu API-d. Teise API saab valida aadressil https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Selgitus:

Siin asub teie pistikprogrammi kood.

Klassi definitsioon: `class BookingPlugin`: Määratleb klassi nimega BookingPlugin, mis sisaldab meetodeid hotellide ja lendude broneerimiseks.

Hotelli broneerimise meetod:

- `@kernel_function(description="booking hotel")`: Dekoraator, mis kirjeldab funktsiooni tuumfunktsioonina hotellide broneerimiseks.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Määratleb meetodi hotellide broneerimiseks koos kommenteeritud parameetrite ja tagastustüübiga.

Meetod koostab sõnastiku parameetritest hotelli broneerimispäringuks ja saadab SERP API-le GET-päringu. Ta kontrollib vastuse staatust ja tagastab hotelli omadused, kui päring õnnestub, või None, kui päring ebaõnnestus.

Lennu broneerimise meetod:

- `@kernel_function(description="booking flight")`: Dekoraator, mis kirjeldab funktsiooni tuumfunktsioonina lendude broneerimiseks.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Määratleb meetodi lendude broneerimiseks koos kommenteeritud parameetrite ja tagastustüübiga.

Meetod koostab sõnastikud parameetritest väljuva ja tagasilennu päringuks ning saadab SERP API-le GET-päringud. Ta kontrollib vastuse staatust ja lisab lendude info tulemustesse, kui päring õnnestub, või prindib veateate, kui päring ebaõnnestus. Meetod tagastab tulemuse, mis sisaldab lendude infot.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Selgitus:
Impordilaused: Impordib vajalikud moodulid Azure'i volituste, AI agendi, vestluste sõnumi sisu, autori rolli ja kerneli funktsiooni dekoratsiooni jaoks.

Asünkroonne kontekstihaldur: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): See seab üles asünkroonse kontekstihalduri, et hallata Azure'i volitusi ja luua AI agendi klient.

Agendi nimi ja juhised:  
- `AGENT_NAME = "BookingAgent"`: Määrab agendi nime.  
- `AGENT_INSTRUCTIONS = """..."""`: Annab agendile üksikasjalikud juhised, kuidas käsitleda broneerimisfoorme.

Loo agendi definitsioon: `agent_definition = await client.agents.create_agent(...)`: Loob agendi definitsiooni määratud mudeli, nime ja juhistega.

Loo AzureAI agent: `agent = AzureAIAgent(...)`: Loob AzureAI agendi, kasutades klienti, agendi definitsiooni ja määratud pistikprogrammi.

Loo lõim: `thread: AzureAIAgentThread | None = None`: Loob agendi jaoks lõime. Lõime loomine ei ole alguses kohustuslik – kui väärtuseks määrata `None`, siis luuakse esimese käivituse ajal uus lõim ja tagastatakse vastuse osana.

Kasutaja sisendid: `user_inputs = ["..."]`: Määrab kasutaja sisendite loendi, mida agent töötleb.

finally plokis kustutatakse lõim ja agent, et vabastada ressursid.


# Autentimine

Klass `DefaultAzureCredential` on osa Azure SDK-st Pythoni jaoks. See pakub vaikimisi viisi Azure teenustega autentimiseks. See proovib autentida mitme meetodi abil kindlas järjekorras, nagu keskkonnamuutujad, hallatud identiteet ja Azure CLI volitused.

Asünkroonsed operatsioonid: moodul aio näitab, et klass DefaultAzureCredential toetab asünkroonseid operatsioone. See tähendab, et saate seda kasutada koos asyncio’ga mitteblokeerivate autentimispäringute tegemiseks.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Lahtiütlus**:  
See dokument on tõlgitud kasutades tehisintellektil põhinevat tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi me püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle algkeeles tuleks pidada autoriteetseks allikaks. Tähtsa teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta ühegi selle tõlke kasutamisest tuleneva arusaamatuse või väärinterpreteerimise eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
